# TEST vs PROD Data Comparison & QA Validation

**Purpose:** Automated validation of RShiny app data between TEST and PROD environments

---

## What This Notebook Does

✅ **Compares data** from `member_explorer` and `zip_explorer` files  
✅ **Validates** member counts, demographics, HEDIS measures, prevalence, SDoH metrics  
✅ **Tracks** data quality (unknown/missing values)  
✅ **Generates** professional HTML reports and CSV exports  
✅ **Provides** interactive QA checklist for sign-off  

**Time saved:** From 1+ day → 15 minutes

---

## Quick Start Guide

### 1. Set Up Your Folders
```
project/
├── data/
│   ├── test/    # Put TEST environment exports here
│   └── prod/    # Put PROD environment exports here
└── output/      # Reports will be saved here (auto-created)
```

### 2. Export Data from RShiny App
Export with this naming pattern:
- `member_explorer_[Plan]_[State]_[Population].csv`
- `zip_explorer_[Plan]_[State]_[Population].csv`

**Example:** `member_explorer_PlanA_CA_Medicare.csv`

### 3. Configure Scenarios
Edit the configuration cell below (Section 3)

### 4. Run the Notebook
- Menu: **Cell → Run All**
- Or: Press **Shift+Enter** on each cell

### 5. Review Results
- Open `output/comparison_report.html` in your browser
- Review CSV files in `output/` folder
- Fill out QA checklist at the end of this notebook

---

## 1. Install Required Packages

**Run this cell once** to install dependencies (takes ~2 minutes first time).

In [ ]:
import sys
!{sys.executable} -m pip install pandas numpy scipy --quiet

print("✓ All packages installed successfully!")

## 2. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")
print(f"   Pandas version: {pd.__version__}")
print(f"   NumPy version: {np.__version__}")

## 3. Configuration

### ⚠️ EDIT THIS CELL TO MATCH YOUR DATA

In [ ]:
# ==============================================================================
# CONFIGURATION - CUSTOMIZE THESE VALUES
# ==============================================================================

# Data paths (relative to this notebook)
TEST_DATA_PATH = Path("data/test")
PROD_DATA_PATH = Path("data/prod")
OUTPUT_PATH = Path("output")

# Create output directory
OUTPUT_PATH.mkdir(exist_ok=True, parents=True)

# ============================================================================
# DEFINE YOUR TEST SCENARIOS
# Add one row for each Plan/State/Population combination you want to test
# ============================================================================

test_scenarios = pd.DataFrame([
    {"scenario_id": "CA_Med_PlanA",   "plan": "PlanA", "state": "CA", "population": "Medicare"},
    {"scenario_id": "CA_Med_PlanB",   "plan": "PlanB", "state": "CA", "population": "Medicare"},
    {"scenario_id": "CA_Mcaid_PlanA", "plan": "PlanA", "state": "CA", "population": "Medicaid"},
    # Add more scenarios below:
    # {"scenario_id": "TX_Med_PlanA", "plan": "PlanA", "state": "TX", "population": "Medicare"},
])

# ============================================================================
# COMPARISON THRESHOLDS
# Adjust these based on what's acceptable for your data
# ============================================================================

THRESHOLDS = {
    'member_count_match_pct': 1,      # < 1% difference = Match
    'member_count_minor_pct': 5,      # 1-5% difference = Minor
    'hedis_match_pp': 0.5,            # < 0.5 percentage points = Match
    'hedis_minor_pp': 2,              # 0.5-2 pp = Minor
    'demographic_match_pp': 1,        # < 1 pp = Match
    'demographic_minor_pp': 3,        # 1-3 pp = Minor
}

# ==============================================================================

print("="*60)
print("CONFIGURATION LOADED")
print("="*60)
print(f"TEST data path: {TEST_DATA_PATH}")
print(f"PROD data path: {PROD_DATA_PATH}")
print(f"Output path: {OUTPUT_PATH}")
print(f"\nScenarios to test: {len(test_scenarios)}")
print("\nScenario list:")
for idx, row in test_scenarios.iterrows():
    print(f"  {idx+1}. {row['scenario_id']}: {row['plan']} / {row['state']} / {row['population']}")
print("="*60)

## 4. Helper Functions

Core comparison logic - **no need to edit this section**

In [ ]:
def load_data(file_name, env, scenario):
    """Load data from TEST or PROD environment"""
    base_path = TEST_DATA_PATH if env == "TEST" else PROD_DATA_PATH
    
    # Construct filename: dataset_plan_state_population.csv
    filename = f"{file_name}_{scenario['plan']}_{scenario['state']}_{scenario['population']}.csv"
    file_path = base_path / filename
    
    if not file_path.exists():
        print(f"   ⚠ File not found: {file_path}")
        return None
    
    # Load and standardize column names
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
    return df

def calc_pct_change(new_val, old_val):
    """Calculate percentage change"""
    if pd.isna(old_val) or old_val == 0:
        return np.nan
    return ((new_val - old_val) / old_val) * 100

def get_status(value, match_threshold, minor_threshold, is_pp=False):
    """Determine status based on thresholds"""
    if pd.isna(value):
        return "⚠ Unknown"
    abs_val = abs(value)
    if abs_val < match_threshold:
        return "✓ Match"
    elif abs_val < minor_threshold:
        return "⚠ Minor Diff"
    else:
        return "✗ Major Diff"

print("✓ Helper functions defined")

In [ ]:
def compare_member_counts(test_df, prod_df, scenario_id):
    """Compare member counts between TEST and PROD"""
    test_count = len(test_df)
    prod_count = len(prod_df)
    
    # If there's a member_count column, sum it instead
    if 'member_count' in test_df.columns:
        test_count = test_df['member_count'].sum()
        prod_count = prod_df['member_count'].sum()
    
    pct_change = calc_pct_change(test_count, prod_count)
    status = get_status(pct_change, 
                       THRESHOLDS['member_count_match_pct'], 
                       THRESHOLDS['member_count_minor_pct'])
    
    return pd.DataFrame([{
        'scenario': scenario_id,
        'metric': 'Total Members',
        'test_value': int(test_count),
        'prod_value': int(prod_count),
        'difference': int(test_count - prod_count),
        'pct_change': round(pct_change, 2) if not pd.isna(pct_change) else np.nan,
        'status': status
    }])

def analyze_demographics(test_df, prod_df, scenario_id):
    """Analyze demographic distribution changes"""
    results = []
    
    # Find demographic columns (common names)
    demo_cols = [col for col in test_df.columns if col in 
                 ['age_group', 'age_band', 'age_category', 'gender', 'sex', 
                  'race', 'race_ethnicity', 'ethnicity', 'ethnic_group', 
                  'county', 'region']]
    
    for col in demo_cols:
        if col not in prod_df.columns:
            continue
            
        # Calculate percentage distributions
        test_dist = test_df[col].value_counts(normalize=True, dropna=False) * 100
        prod_dist = prod_df[col].value_counts(normalize=True, dropna=False) * 100
        
        # Compare each category
        all_categories = set(test_dist.index) | set(prod_dist.index)
        
        for category in all_categories:
            test_pct = test_dist.get(category, 0)
            prod_pct = prod_dist.get(category, 0)
            pp_change = test_pct - prod_pct
            
            # Determine status
            if test_pct == 0 and prod_pct > 0:
                status = "✗ Missing in TEST"
            elif test_pct > 0 and prod_pct == 0:
                status = "✗ New in TEST"
            else:
                status = get_status(abs(pp_change), 
                                  THRESHOLDS['demographic_match_pp'], 
                                  THRESHOLDS['demographic_minor_pp'])
            
            results.append({
                'scenario': scenario_id,
                'demographic': col,
                'category': str(category),
                'test_pct': round(test_pct, 2),
                'prod_pct': round(prod_pct, 2),
                'pct_point_change': round(pp_change, 2),
                'status': status
            })
    
    return pd.DataFrame(results)

def compare_unknown_proportions(test_df, prod_df, scenario_id):
    """Track unknown/missing values for data quality"""
    results = []
    
    # Find categorical columns
    cat_cols = test_df.select_dtypes(include=['object']).columns
    
    for col in cat_cols:
        if col not in prod_df.columns:
            continue
        
        # Calculate % unknown in each environment
        unknown_keywords = ['unknown', 'missing', 'null', 'na', 'not specified', 
                          'unspecified', 'blank', 'none', '']
        
        test_total = len(test_df)
        prod_total = len(prod_df)
        
        test_unknown = sum(test_df[col].fillna('').astype(str).str.lower().str.contains('|'.join(unknown_keywords)))
        prod_unknown = sum(prod_df[col].fillna('').astype(str).str.lower().str.contains('|'.join(unknown_keywords)))
        
        test_unknown_pct = (test_unknown / test_total * 100) if test_total > 0 else 0
        prod_unknown_pct = (prod_unknown / prod_total * 100) if prod_total > 0 else 0
        
        # Only include if there are unknowns
        if test_unknown_pct > 0 or prod_unknown_pct > 0:
            change = test_unknown_pct - prod_unknown_pct
            
            if change < 0:
                status = "✓ Improved"
            elif change == 0:
                status = "→ No Change"
            else:
                status = "✗ Worsened"
            
            results.append({
                'scenario': scenario_id,
                'field': col,
                'test_unknown_pct': round(test_unknown_pct, 2),
                'prod_unknown_pct': round(prod_unknown_pct, 2),
                'change': round(change, 2),
                'status': status
            })
    
    return pd.DataFrame(results)

def compare_zip_metrics(test_df, prod_df, scenario_id):
    """Compare zip-level metrics (HEDIS, prevalence, SDoH)"""
    results = {'summary': None, 'metrics': []}
    
    # Zip code coverage
    test_zips = test_df['zip_code'].nunique() if 'zip_code' in test_df.columns else 0
    prod_zips = prod_df['zip_code'].nunique() if 'zip_code' in prod_df.columns else 0
    
    pct_change = calc_pct_change(test_zips, prod_zips)
    status = "✓ Match" if test_zips == prod_zips else "✗ Different"
    
    results['summary'] = pd.DataFrame([{
        'scenario': scenario_id,
        'metric': 'Unique Zip Codes',
        'test_value': test_zips,
        'prod_value': prod_zips,
        'difference': test_zips - prod_zips,
        'pct_change': round(pct_change, 2) if not pd.isna(pct_change) else np.nan,
        'status': status
    }])
    
    # Compare numeric metrics
    numeric_cols = test_df.select_dtypes(include=[np.number]).columns
    # Exclude ID columns
    numeric_cols = [col for col in numeric_cols if col not in ['zip_code', 'fips', 'fips_code', 'member_count']]
    
    for col in numeric_cols:
        if col not in prod_df.columns:
            continue
        
        test_avg = test_df[col].mean()
        prod_avg = prod_df[col].mean()
        test_complete = test_df[col].notna().sum()
        prod_complete = prod_df[col].notna().sum()
        
        avg_pct_change = calc_pct_change(test_avg, prod_avg)
        
        results['metrics'].append({
            'scenario': scenario_id,
            'metric': col,
            'test_avg': round(test_avg, 2) if not pd.isna(test_avg) else np.nan,
            'prod_avg': round(prod_avg, 2) if not pd.isna(prod_avg) else np.nan,
            'test_complete': test_complete,
            'prod_complete': prod_complete,
            'avg_diff': round(test_avg - prod_avg, 2) if not pd.isna(test_avg) and not pd.isna(prod_avg) else np.nan,
            'avg_pct_change': round(avg_pct_change, 2) if not pd.isna(avg_pct_change) else np.nan,
            'completeness_change': test_complete - prod_complete
        })
    
    results['metrics'] = pd.DataFrame(results['metrics'])
    return results

def compare_datasets(test_df, prod_df, dataset_name, scenario_id):
    """High-level dataset structure comparison"""
    return pd.DataFrame([{
        'scenario': scenario_id,
        'dataset': dataset_name,
        'test_rows': len(test_df),
        'prod_rows': len(prod_df),
        'row_diff': len(test_df) - len(prod_df),
        'test_cols': len(test_df.columns),
        'prod_cols': len(prod_df.columns),
        'col_diff': len(test_df.columns) - len(prod_df.columns)
    }])

print("✓ Comparison functions defined")

## 5. Run Comparisons

This cell processes all scenarios and generates comparison results.

In [ ]:
print("\n" + "="*70)
print("STARTING TEST vs PROD COMPARISON")
print("="*70 + "\n")

# Initialize result storage
all_results = {
    'member_counts': [],
    'demographics': [],
    'unknown_proportions': [],
    'zip_summary': [],
    'zip_metrics': [],
    'dataset_comparison': []
}

# Process each scenario
for idx, scenario in test_scenarios.iterrows():
    scenario_id = scenario['scenario_id']
    
    print(f"\n{'─'*70}")
    print(f"Processing: {scenario_id}")
    print(f"  Plan: {scenario['plan']} | State: {scenario['state']} | Population: {scenario['population']}")
    print(f"{'─'*70}\n")
    
    # ---- MEMBER EXPLORER ----
    print("Loading member_explorer...", end=" ")
    test_member = load_data("member_explorer", "TEST", scenario)
    prod_member = load_data("member_explorer", "PROD", scenario)
    
    if test_member is not None and prod_member is not None:
        print("✓")
        
        # Member counts
        all_results['member_counts'].append(
            compare_member_counts(test_member, prod_member, scenario_id)
        )
        
        # Demographics
        demo_results = analyze_demographics(test_member, prod_member, scenario_id)
        if not demo_results.empty:
            all_results['demographics'].append(demo_results)
        
        # Unknown proportions
        unknown_results = compare_unknown_proportions(test_member, prod_member, scenario_id)
        if not unknown_results.empty:
            all_results['unknown_proportions'].append(unknown_results)
        
        # Dataset comparison
        all_results['dataset_comparison'].append(
            compare_datasets(test_member, prod_member, "member_explorer", scenario_id)
        )
    else:
        print("✗ SKIPPED")
    
    # ---- ZIP EXPLORER ----
    print("Loading zip_explorer...", end=" ")
    test_zip = load_data("zip_explorer", "TEST", scenario)
    prod_zip = load_data("zip_explorer", "PROD", scenario)
    
    if test_zip is not None and prod_zip is not None:
        print("✓")
        
        # Zip metrics
        zip_results = compare_zip_metrics(test_zip, prod_zip, scenario_id)
        if zip_results['summary'] is not None:
            all_results['zip_summary'].append(zip_results['summary'])
        if not zip_results['metrics'].empty:
            all_results['zip_metrics'].append(zip_results['metrics'])
        
        # Dataset comparison
        all_results['dataset_comparison'].append(
            compare_datasets(test_zip, prod_zip, "zip_explorer", scenario_id)
        )
    else:
        print("✗ SKIPPED")

# Combine all results
for key in all_results:
    if all_results[key]:
        all_results[key] = pd.concat(all_results[key], ignore_index=True)
    else:
        all_results[key] = pd.DataFrame()

print("\n" + "="*70)
print("✓ ALL COMPARISONS COMPLETE")
print("="*70)

## 6. Results Summary

View quick summary of key findings:

In [ ]:
print("\n" + "="*70)
print("RESULTS SUMMARY")
print("="*70 + "\n")

# Member Counts
if not all_results['member_counts'].empty:
    print("📊 MEMBER COUNT CHANGES")
    print("─"*70)
    display(all_results['member_counts'])
    print()

# Unknown Proportions
if not all_results['unknown_proportions'].empty:
    print("🔍 DATA QUALITY: Unknown/Missing Values")
    print("─"*70)
    display(all_results['unknown_proportions'])
    print()

# Zip Coverage
if not all_results['zip_summary'].empty:
    print("📍 ZIP CODE COVERAGE")
    print("─"*70)
    display(all_results['zip_summary'])
    print()

# Dataset Structure
if not all_results['dataset_comparison'].empty:
    print("📋 DATASET STRUCTURE")
    print("─"*70)
    display(all_results['dataset_comparison'])
    print()

## 7. Detailed Results - Demographics

View demographic distribution changes (filtered to show significant changes only):

In [ ]:
if not all_results['demographics'].empty:
    # Show only changes > 1 percentage point
    significant_demo = all_results['demographics'][
        abs(all_results['demographics']['pct_point_change']) > 1
    ]
    
    if not significant_demo.empty:
        print("📈 SIGNIFICANT DEMOGRAPHIC CHANGES (>1 percentage point)")
        print("="*70)
        display(significant_demo.sort_values('pct_point_change', key=abs, ascending=False))
    else:
        print("✓ All demographic distributions within 1% of PROD")
else:
    print("No demographic data to display")

## 8. Detailed Results - Zip Metrics

View zip-level metric changes (HEDIS, prevalence, SDoH):

In [ ]:
if not all_results['zip_metrics'].empty:
    # Show metrics with > 5% change or completeness issues
    significant_zip = all_results['zip_metrics'][
        (abs(all_results['zip_metrics']['avg_pct_change']) > 5) |
        (all_results['zip_metrics']['completeness_change'] != 0)
    ]
    
    if not significant_zip.empty:
        print("📊 SIGNIFICANT ZIP METRIC CHANGES (>5% or completeness issues)")
        print("="*70)
        display(significant_zip[['scenario', 'metric', 'test_avg', 'prod_avg', 
                                 'avg_pct_change', 'completeness_change']].sort_values(
            'avg_pct_change', key=abs, ascending=False
        ))
    else:
        print("✓ All zip-level metrics stable (within 5%)")
else:
    print("No zip metrics data to display")

## 9. Export Results

Save results to CSV files and generate HTML report:

In [ ]:
print("\nExporting results...\n")

# Export CSVs
exports = [
    ('member_counts', 'member_counts_comparison.csv'),
    ('demographics', 'demographics_comparison.csv'),
    ('unknown_proportions', 'unknown_proportions.csv'),
    ('zip_summary', 'zip_coverage.csv'),
    ('zip_metrics', 'zip_metrics_comparison.csv'),
    ('dataset_comparison', 'dataset_structure.csv')
]

for key, filename in exports:
    if not all_results[key].empty:
        filepath = OUTPUT_PATH / filename
        all_results[key].to_csv(filepath, index=False)
        print(f"✓ Saved: {filepath}")

print("\n" + "="*70)
print("✓ ALL EXPORTS COMPLETE")
print(f"📁 Files saved to: {OUTPUT_PATH.absolute()}")
print("="*70)

## 10. Generate HTML Report

Create professional HTML report:

In [ ]:
from datetime import datetime

# Generate HTML report
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>TEST vs PROD Comparison Report</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 40px;
            background: #f5f7fa;
            color: #2d3748;
        }}
        .container {{ max-width: 1400px; margin: 0 auto; }}
        h1 {{
            color: #1a202c;
            border-bottom: 4px solid #4299e1;
            padding-bottom: 15px;
            font-size: 2.5em;
        }}
        h2 {{
            color: #2d3748;
            margin-top: 40px;
            padding: 15px;
            background: white;
            border-left: 5px solid #4299e1;
            box-shadow: 0 1px 3px rgba(0,0,0,0.1);
        }}
        .summary {{
            background: white;
            padding: 25px;
            border-radius: 8px;
            margin: 20px 0;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        }}
        .header-info {{
            background: #edf2f7;
            padding: 20px;
            border-radius: 8px;
            margin: 20px 0;
        }}
        table {{
            border-collapse: collapse;
            width: 100%;
            margin: 20px 0;
            background: white;
            font-size: 0.95em;
        }}
        th {{
            background: #2d3748;
            color: white;
            padding: 14px;
            text-align: left;
            font-weight: 600;
        }}
        td {{
            padding: 12px 14px;
            border-bottom: 1px solid #e2e8f0;
        }}
        tr:hover {{ background: #f7fafc; }}
        .status-match {{ color: #38a169; font-weight: bold; }}
        .status-minor {{ color: #d69e2e; font-weight: bold; }}
        .status-major {{ color: #e53e3e; font-weight: bold; }}
    </style>
</head>
<body>
    <div class='container'>
        <h1>📊 TEST vs PROD Comparison Report</h1>
        <div class='header-info'>
            <p><strong>Generated:</strong> {timestamp}</p>
            <p><strong>Scenarios Tested:</strong> {len(test_scenarios)}</p>
        </div>
"""

# Member Counts
if not all_results['member_counts'].empty:
    html_content += "<h2>👥 Member Count Summary</h2><div class='summary'>"
    html_content += all_results['member_counts'].to_html(index=False, classes='table', escape=False)
    html_content += "</div>"

# Unknown Proportions
if not all_results['unknown_proportions'].empty:
    html_content += "<h2>🔍 Data Quality: Unknown/Missing Values</h2><div class='summary'>"
    html_content += all_results['unknown_proportions'].to_html(index=False, classes='table', escape=False)
    html_content += "</div>"

# Demographics (significant changes only)
if not all_results['demographics'].empty:
    significant_demo = all_results['demographics'][abs(all_results['demographics']['pct_point_change']) > 1]
    if not significant_demo.empty:
        html_content += "<h2>📈 Demographic Distribution Changes (>1pp)</h2><div class='summary'>"
        html_content += significant_demo.to_html(index=False, classes='table', escape=False)
        html_content += "</div>"

# Zip Summary
if not all_results['zip_summary'].empty:
    html_content += "<h2>📍 Zip Code Coverage</h2><div class='summary'>"
    html_content += all_results['zip_summary'].to_html(index=False, classes='table', escape=False)
    html_content += "</div>"

# Dataset Structure
if not all_results['dataset_comparison'].empty:
    html_content += "<h2>📋 Dataset Structure Comparison</h2><div class='summary'>"
    html_content += all_results['dataset_comparison'].to_html(index=False, classes='table', escape=False)
    html_content += "</div>"

html_content += """
    </div>
</body>
</html>
"""

# Save HTML report
report_path = OUTPUT_PATH / "comparison_report.html"
with open(report_path, 'w') as f:
    f.write(html_content)

print(f"\n✓ HTML Report generated: {report_path.absolute()}")
print(f"\n📂 Open this file in your browser to view the full report")

---

# QA VALIDATION CHECKLIST

Use this section to document your review and sign-off.

---

## Release Information

**Release Date:** _________________________  
**Validated By:** _________________________  
**Validation Date:** _________________________  

---

## Pre-Validation Checklist

Run this cell to create an interactive checklist:

In [ ]:
from IPython.display import HTML, display

checklist_html = """
<style>
    .checklist { background: white; padding: 20px; border-radius: 5px; margin: 10px 0; }
    .checklist h3 { color: #2d3748; border-bottom: 2px solid #4299e1; padding-bottom: 10px; }
    .checklist-item { padding: 8px 0; }
    .checklist-item input { margin-right: 10px; transform: scale(1.2); }
    .notes { width: 100%; padding: 10px; margin: 10px 0; border: 1px solid #cbd5e0; border-radius: 4px; }
</style>

<div class="checklist">
    <h3>☑️ Pre-Validation Checklist</h3>
    <div class="checklist-item"><input type="checkbox"> All TEST environment data exported</div>
    <div class="checklist-item"><input type="checkbox"> All PROD environment data exported</div>
    <div class="checklist-item"><input type="checkbox"> Data exports are from same time period</div>
    <div class="checklist-item"><input type="checkbox"> All required scenarios covered</div>
    <div class="checklist-item"><input type="checkbox"> Comparison script executed successfully</div>
</div>

<div class="checklist">
    <h3>📊 Member Count Validation</h3>
    <div class="checklist-item"><input type="checkbox"> Total member counts within acceptable range (< ±5%)</div>
    <div class="checklist-item"><input type="checkbox"> Member count changes documented and explained</div>
    <div class="checklist-item"><input type="checkbox"> No unexpected member additions/removals</div>
    <p><strong>Notes:</strong></p>
    <textarea class="notes" rows="3" placeholder="Document any member count changes and explanations..."></textarea>
</div>

<div class="checklist">
    <h3>🔍 Data Quality: Unknown/Missing Values</h3>
    <div class="checklist-item"><input type="checkbox"> Unknown race proportion: Reduced or stable</div>
    <div class="checklist-item"><input type="checkbox"> Unknown gender proportion: Reduced or stable</div>
    <div class="checklist-item"><input type="checkbox"> Unknown ethnicity proportion: Reduced or stable</div>
    <div class="checklist-item"><input type="checkbox"> Unknown age group proportion: Reduced or stable</div>
    <p><em>Target: All unknown categories should decrease or remain ≤ prior release</em></p>
    <p><strong>Notes:</strong></p>
    <textarea class="notes" rows="3" placeholder="Document any data quality issues or improvements..."></textarea>
</div>

<div class="checklist">
    <h3>📈 HEDIS Measures (29 measures)</h3>
    <div class="checklist-item"><input type="checkbox"> All 29 HEDIS measures present in both environments</div>
    <div class="checklist-item"><input type="checkbox"> No measure rates differ by > 2 percentage points</div>
    <div class="checklist-item"><input type="checkbox"> Any differences have documented explanations</div>
    <div class="checklist-item"><input type="checkbox"> Performance improvements validated</div>
    <p><strong>Measures with significant changes:</strong></p>
    <textarea class="notes" rows="4" placeholder="List any HEDIS measures with changes > 1pp and explanations..."></textarea>
</div>

<div class="checklist">
    <h3>🗺️ Geographic Coverage</h3>
    <div class="checklist-item"><input type="checkbox"> All expected zip codes present</div>
    <div class="checklist-item"><input type="checkbox"> County-level aggregations match</div>
    <div class="checklist-item"><input type="checkbox"> No missing geographic data</div>
</div>

<div class="checklist">
    <h3>📋 SDoH Measures</h3>
    <div class="checklist-item"><input type="checkbox"> All zip codes have SDoH data</div>
    <div class="checklist-item"><input type="checkbox"> SDoH measures within expected ranges</div>
    <div class="checklist-item"><input type="checkbox"> No missing or null values</div>
    <div class="checklist-item"><input type="checkbox"> Metrics properly calculated</div>
</div>

<div class="checklist">
    <h3>⚠️ Known Issues</h3>
    <p><strong>Issue #1:</strong></p>
    <textarea class="notes" rows="2" placeholder="Description..."></textarea>
    <p>Impact: <input type="radio" name="impact1"> Low <input type="radio" name="impact1"> Medium <input type="radio" name="impact1"> High</p>
    <p>Resolution: <input type="radio" name="res1"> Proceed <input type="radio" name="res1"> Fix Required <input type="radio" name="res1"> Under Investigation</p>
    
    <p><strong>Issue #2:</strong></p>
    <textarea class="notes" rows="2" placeholder="Description..."></textarea>
    <p>Impact: <input type="radio" name="impact2"> Low <input type="radio" name="impact2"> Medium <input type="radio" name="impact2"> High</p>
    <p>Resolution: <input type="radio" name="res2"> Proceed <input type="radio" name="res2"> Fix Required <input type="radio" name="res2"> Under Investigation</p>
</div>

<div class="checklist">
    <h3>✅ Final Sign-Off</h3>
    <p><strong>Decision:</strong></p>
    <div class="checklist-item"><input type="radio" name="decision"> <strong>APPROVED</strong> for PROD deployment</div>
    <div class="checklist-item"><input type="radio" name="decision"> <strong>APPROVED</strong> with documented exceptions</div>
    <div class="checklist-item"><input type="radio" name="decision"> <strong>NOT APPROVED</strong> - Issues must be resolved</div>
    
    <p><strong>Exceptions/Caveats:</strong></p>
    <textarea class="notes" rows="3" placeholder="Document any exceptions or conditions for approval..."></textarea>
    
    <p><strong>Signatures:</strong></p>
    <p>QA Validator: _________________________ Date: _____________</p>
    <p>Data Team Lead: _________________________ Date: _____________</p>
    <p>Product Owner: _________________________ Date: _____________</p>
</div>
"""

display(HTML(checklist_html))

---

## Summary Statistics for Checklist

Use these cells to quickly assess overall validation status:

In [ ]:
# Count status types across all results
print("VALIDATION STATUS SUMMARY")
print("="*50)

if not all_results['member_counts'].empty:
    print("\nMember Count Status:")
    print(all_results['member_counts']['status'].value_counts())

if not all_results['unknown_proportions'].empty:
    print("\nData Quality Status:")
    print(all_results['unknown_proportions']['status'].value_counts())

if not all_results['demographics'].empty:
    print("\nDemographic Changes Status:")
    print(all_results['demographics']['status'].value_counts())

---

## Next Steps

1. ✅ Review the HTML report: `output/comparison_report.html`
2. ✅ Examine CSV files in `output/` folder for detailed analysis
3. ✅ Fill out the QA checklist above
4. ✅ Document any issues or exceptions
5. ✅ Get required sign-offs
6. ✅ Archive this notebook with date stamp for audit trail
7. ✅ Deploy to PROD (if approved)

---

## Tips

- **Save this notebook** with a date stamp: `TEST_PROD_Comparison_2024_03_15.ipynb`
- **Print to PDF** for physical sign-off: File → Print Preview → Save as PDF
- **Re-run anytime** to compare new exports
- **Share results** by sending the HTML report via email

---

*Generated with TEST vs PROD Comparison Tool v2.0*